In [10]:
import cv2
import numpy as np
import heapq

In [7]:
video_path = "data/Data/kaniula1.avi"

cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()

cv2.imshow("First Frame", frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

cap.release()

In [20]:
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (15, 15), 0)
edges = cv2.Canny(blurred, 5, 10, 3)

mask = np.zeros_like(edges)
h, w = mask.shape
cv2.circle(mask, (w // 2, h // 2), min(h, w) // 2, 255, -1)

edges = cv2.bitwise_and(edges, mask)

edges_neg = 255 - edges

cv2.imshow("Original", frame)
cv2.imshow("Blurred", blurred)
cv2.imshow("Canny Edges", edges)
cv2.imshow("Negative Edges", edges_neg)

cv2.waitKey(0)
cv2.destroyAllWindows()

cap.release()

In [27]:
def dijkstra_contour(image, LEFT, RIGHT):
    # Apply Canny edge detection
    edges = cv2.Canny(image, 50, 150)

    # Create priority queue
    priority_queue = []

    # Parameters
    height, width = edges.shape
    primary_mask = np.ones_like(edges, dtype=np.uint8)
    primary_mask[height // 2:, :] = 0  # Lower half of the frame
    primary_mask[:, :LEFT] = 0  # Left of the region of interest
    primary_mask[:, RIGHT:] = 0  # Right of the region of interest

    # Initialize distances
    distances = np.full_like(edges, 255, dtype=np.uint8)
    distances[edges == 255] = 0  # White pixels get a level of 255

    # Initialize priority queue with pixels in the lower half of the frame and within the primary mask
    for y in range(height // 2, height):
        for x in range(width):
            if primary_mask[y, x] == 1:
                heapq.heappush(priority_queue, (distances[y, x], (y, x)))

    # Dijkstra's algorithm for bottom contour
    while priority_queue:
        dist, (y, x) = heapq.heappop(priority_queue)

        if x == LEFT:
            break  # Stop when reaching the LEFT parameter

        # Update neighbors
        neighbors = [(y + dy, x + dx) for dy, dx in [(-1, 0), (0, -1), (0, 1), (1, 0)] if 0 <= y + dy < height and 0 <= x + dx < width]
        for ny, nx in neighbors:
            new_dist = dist + distances[ny, nx]
            if new_dist < distances[ny, nx]:
                distances[ny, nx] = new_dist
                heapq.heappush(priority_queue, (new_dist, (ny, nx)))

    # Reconstruct path to the right border of the primary mask
    # You can customize this part based on your needs

    return distances


In [33]:
LEFT_PARAM = 100  # Replace with your actual parameter
RIGHT_PARAM = 400  # Replace with your actual parameter

result = dijkstra_contour(frame, LEFT_PARAM, RIGHT_PARAM)

# Visualize the result or save it
cv2.imshow('Result', result)
cv2.waitKey(0)
cv2.destroyAllWindows()